## Leetcode 743

// Pseudo code for Bellman-Ford algo.

function BellmanFord (list vertices, list edges, vertex source) is

    ::distance[], predecessor[]

    // This implementation takes in a graph, represented as
    // lists of vertices and edges, and fills two arrays
    // (distance and predecessor) about the shortest path
    // from the source to each vertex

    // Step 1: initialize graph
    for each vertex v in vertices do
        distance[v] := inf             // Initialize the distance to all vertices to infinity
        predecessor[v] := null         // And having a null predecessor
    
    distance[source] := 0              // The distance from the source to itself is zero

    // Step 2: relax edges repeatedly
    for i from 1 to size(vertices)−1 do // Just |V|−1 repetitions; i is never referenced
        for each edge (u, v) with weight w in edges do
            if distance[u] + w < distance[v] then
                distance[v] := distance[u] + w
                predecessor[v] := u

    // Step 3: check for negative-weight cycles
    for each edge (u, v) with weight w in edges do
        if distance[u] + w < distance[v] then
            error "Graph contains a negative-weight cycle"

    return distance[], predecessor[]

In [8]:
def networkDelayTime(self, times, N: int, K: int) -> int:
        distance = [float('inf')] * (N+1)
        # predecessor = [None] * (N+1)
        
        distance[K] = 0
        
        for _ in range(N-1):
            for u, v, w in times:
                if distance[u] + w < distance[v]:
                    distance[v] = distance[u] + w
                    # predecessor[v] = u
                    
        # #check for negative weight            
        # for u, v, w in times:
        #     if distance[u] + w < distance[v]:
        #         raise ValueError
                
        return max(distance[1:]) if max(distance[1:]) < float('inf') else -1

In fact this problem can be done with simple Dijkstra. Alternatively we can use bfs or dfs, to avoid reaching worst case (that is, visit all edges) everytime.

In [9]:
def networkDelayTime(self, times, N: int, K: int) -> int:
        graph = collections.defaultdict(list)
        for u, v, w in times:
            graph[u].append((w, v))
            
        distance = {u: float('inf') for u in range(1, N + 1)}
        
        def dfs(vertex, elapsed):
            if elapsed >= distance[vertex]:
                return
            distance[vertex] = elapsed
            for time, neighbor in sorted(graph[vertex]):
                dfs(neighbor, elapsed + time)
                
        dfs(K, 0)
        ans = max(distance.values())
        
        return ans if ans < float('inf') else -1

Note that we do sort in each dfs call, which is costly. Can use heap to improve.

In [10]:
import heapq
def networkDelayTime(self, times, N: int, K: int) -> int:
    graph = collections.defaultdict(list)
    for u, v, w in times:
        graph[u].append((v, w))

    q = [(0, K)]
    distance = {}
    while q:
        d, vert = heapq.heappop(q)
        if vert in distance:
            continue
        distance[vert] = d
        for n, w in graph[vert]:
            if n in distance:
                continue
            heapq.heappush(q, (d+w, n))

    return max(distance.values()) if len(distance) == N else -1

## Leetcode 787

BFS might be the easiest solution.

In [11]:
import heapq, collections
def findCheapestPrice(self, n: int, flights, src: int, dst: int, K: int) -> int:
    graph = collections.defaultdict(list)
    for u, v, w in flights:
        graph[u].append((v, w))

    visited = {}  
    q = collections.deque([(src, 0)])
    visited[src] = 0

    # print(graph, q)  

    stop = 0
    while q and stop <= K:
        size = len(q)
        temp = visited
        for _ in range(size):
            vert, price = q.popleft()
            for neighbor, next_price in graph[vert]:
                # print(vert, neighbor, q, stop)
                if neighbor in visited and price + next_price >= visited[neighbor]:
                    continue
                q.append((neighbor, next_price + price))
                temp[neighbor] = next_price + price
        stop += 1
        visited = temp
        # print(visited)
    return visited[dst] if dst in visited.keys() else -1        

No harm to try Dijkstra though.

In [12]:
import heapq, collections
def findCheapestPrice(self, n: int, flights, src: int, dst: int, K: int) -> int:
    graph = collections.defaultdict(list)
    for u, v, w in flights:
        graph[u].append((v, w))

    dist = [float('inf')] * n
    currstop = [0] * n
    dist[src] = 0
    currstop[src] = 0

    myheap = [(0, 0, src)]
    while myheap:
        cost, stop, vert = heapq.heappop(myheap)
        if vert == dst:
            return cost
        if stop > K:
            continue

        for neighbor, price in graph[vert]:
            if cost + price < dist[neighbor]:
                dist[neighbor] = cost + price
            elif stop < currstop[neighbor]:
                currstop[neighbor] = stop
            heapq.heappush(myheap, (cost + price, stop + 1, neighbor))

    return dist[dst] if dist[dst] != float('inf') else -1